In [1]:
import pandas as pd
import numpy as np
import os

In [9]:
datalocation = './datasets.uwf.edu/data/UWF-ZeekData22/'

cols = ['resp_pkts', 
        'service', 
        'orig_ip_bytes', 
        'local_resp', 
        'missed_bytes',
        'protocol', 
        'duration', 
        'conn_state', 
        'dest_ip', 
        'orig_pkts',
        'community_id', 
        'resp_ip_bytes', 
        'dest_port', 
        'orig_bytes',
        'local_orig', 
        'datetime', 
        'history', 
        'resp_bytes', 
        'uid', 
        'src_port',
        'ts', 
        'src_ip', 
        'mitre_attack_tactics'
        ]

df = pd.DataFrame(columns=cols)

for root, dirs, files in os.walk(datalocation):
    for name in files:
        if name.endswith('.csv'):
            dfi = pd.read_csv(os.path.join(root, name))       
        elif name.endswith('.parquet'):
            dfi = pd.read_parquet(os.path.join(root, name), engine='pyarrow')
            
        else:
          continue
        if dfi.shape[1] != 23:
          print("wrong shape: ", os.path.join(name))
          continue
        print(os.path.join(name))
        df =  pd.DataFrame(np.concatenate([df.values, dfi.values]), columns=cols)
print(df.shape)
        

part-00000-36d29ab0-aecd-4e34-8d7a-cf9b94be3fea-c000.csv
part-00000-0af89d10-df53-44fd-b124-a8a496fd5023-c000.csv
part-00000-8283e5f3-ac2e-4523-a7f0-f98f0c7d879f-c000.csv
part-00000-5b4f5c3f-e8a9-4020-8fa1-e8985f7c27f3-c000.csv
part-00000-bd55a170-4439-45b0-9122-9a6b9da9760e-c000.csv
part-00000-318611a1-7cdc-4dd0-9348-c6368917fd0c-c000.csv
part-00000-95e0a460-e7c5-4b35-8367-c2e6fbbcf9e1-c000.csv
part-00000-0f7bf56c-ed34-4235-a8d4-1c86eac7ff66-c000.csv
part-00000-15e3dd03-ea76-429e-a52a-ce96a90517f9-c000.csv
part-00000-12352875-21a6-4bd8-aaf1-922e121ef289-c000.csv
part-00000-26e9208e-7819-451b-b23f-2e47f6d1e834-c000.snappy.parquet
wrong shape:  part-00000-d32a9d5e-45b7-4e51-807e-1af297aba2df-c000.csv
part-00000-df678a79-4a73-452b-8e72-d624b2732f17-c000.snappy.parquet
part-00000-b1a9fc13-8068-4a5d-91b2-871438709e81-c000.snappy.parquet
part-00000-7c2e9adb-5430-4792-a42b-10ff5bbd46e8-c000.snappy.parquet
part-00000-3f86626a-1225-47f9-a5a2-0170b737e404-c000.snappy.parquet
part-00000-cbf26680

In [11]:
#Calculate edges
dur = df['duration'].copy()
numdur = dur[dur.notnull()].astype(float)
stddev = numdur.std()
mean = numdur.mean()

if mean - stddev * 2 < 0:
    while mean-2 * stddev < 0:
        mean += stddev

edgena = float('-inf')
edge0 = 0
edge1 = mean - stddev * 2
edge2 = mean - stddev
edge3 = mean
edge4 = mean + stddev
edge5 = mean + stddev * 2
edge6 = float('inf')
edges = pd.IntervalIndex.from_arrays([edgena,edge0, edge1, edge2, edge3, edge4, edge5], [edge0, edge1, edge2, edge3, edge4, edge5, edge6])
dur[dur.notnull() == False] = -10000
cutdur = pd.cut(dur.astype(float), edges, labels=list(range(7)))
cutdur.value_counts()

IntervalIndex([(-inf, 0.0], (0.0, 1.6640604440353854], (1.6640604440353854, 1310.827961175075], (1310.827961175075, 2619.9918619061145], (2619.9918619061145, 3929.155762637154], (3929.155762637154, 5238.319663368194], (5238.319663368194, inf]], dtype='interval[float64, right]')

In [22]:
df["duration"].loc[df["duration"].notnull() == False] = -10000
df['duration'] = pd.qcut(df['duration'].astype(float), q=7, labels=list(range(7)))